## 1. Install requirements

In [8]:
pip install selenium webdriver-manager pandas


Note: you may need to restart the kernel to use updated packages.


## Python Selenium Required Libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time


## 3 Setup Chrome Driver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

driver = webdriver.Chrome(
    service=webdriver.chrome.service.Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 20)


## 4 Open Swiggy Instamart

In [3]:
driver.get("https://www.swiggy.com/instamart")
time.sleep(5)


## 5 Function to Set Pincode



In [4]:
def set_pincode(pincode):
    time.sleep(5)

    # Click the search location box (DIV, not input)
    search_box = wait.until(
        EC.element_to_be_clickable((
            By.XPATH, "//div[@data-testid='search-location']"
        ))
    )

    driver.execute_script("arguments[0].click();", search_box)
    time.sleep(2)

    #  Type using active element (Swiggy creates hidden input dynamically)
    active = driver.switch_to.active_element
    active.send_keys(pincode)
    time.sleep(2)
    active.send_keys(Keys.ENTER)

    time.sleep(8)


set_pincode("110014")

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def set_pincode(pincode):
    # wait for the input field
    search_input = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//input[contains(@placeholder,'Search for area')]"))
    )
    search_input.clear()
    search_input.send_keys(pincode)
    time.sleep(2)  # small wait for suggestions to appear

    # select the first suggestion that contains the pin code
    first_result = wait.until(
        EC.element_to_be_clickable((
            By.XPATH, f"//div[contains(text(),'{pincode}')]"
        ))
    )
    first_result.click()
    time.sleep(5)

set_pincode("110014")


In [6]:
# Confirm Location button
confirm_btn = wait.until(
    EC.element_to_be_clickable((
        By.XPATH, "//span[normalize-space()='Confirm Location']/ancestor::button"
    ))
)

confirm_btn.click()


## keyword 


In [7]:
from selenium.webdriver.common.action_chains import ActionChains

def search_keyword(keyword):
    time.sleep(2)

    # Click search container
    search_box = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//button[@data-testid='search-container']")
        )
    )
    driver.execute_script("arguments[0].click();", search_box)
    time.sleep(2)

    actions = ActionChains(driver)

    # CTRL + A (select all) → BACKSPACE
    actions.key_down(Keys.CONTROL)\
           .send_keys("a")\
           .key_up(Keys.CONTROL)\
           .send_keys(Keys.BACKSPACE)\
           .perform()

    time.sleep(1)

    # Type keyword and ENTER
    actions.send_keys(keyword).send_keys(Keys.ENTER).perform()

    # Wait for results
    time.sleep(6)


## Call Search function 

In [ ]:
search_keyword("Shampoo")


## Page ready for scape 

In [ ]:
products = driver.find_elements(By.XPATH, "//div[@data-testid='product-card']")

print("Products found:", len(products))


## Scroll to Load Products


In [ ]:
def scroll_page(scrolls=5):
    for i in range(scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

scroll_page(6)


## Product Cards

In [ ]:
product_cards = driver.find_elements(By.XPATH, "//div[@data-testid='product-card']")
print("Total products loaded:", len(product_cards))


## Extract Product Details

In [ ]:
data = []

SEARCH_KEYWORD = "Shampoo"
PINCODE = "110014"

for card in product_cards:
    try:
        name = card.find_element(By.XPATH, ".//div[contains(@class,'ProductName')]").text
    except:
        name = None

    try:
        price = card.find_element(By.XPATH, ".//span[contains(text(),'₹')]").text
    except:
        price = None

    # Brand logic (basic)
    if name:
        brand = name.split(" ")[0]
    else:
        brand = None

    data.append({
        "product_name": name,
        "brand": brand,
        "price": price,
        "platform": "Instamart",
        "search_keyword": SEARCH_KEYWORD,
        "pincode": PINCODE
    })


## Convert to Dataframe 



In [ ]:
df = pd.DataFrame(data)
df.head()
